In [ ]:
from PIL import Image
import sys
import pandas as pd
import os
import cv2
import numpy as np
import time


In [ ]:
###
# Input the file names
# @input startTime - Start video from a certain time-frame 00:00:00.000 (hours,minutes,seconds,milliseconds), user-determined
# @input videoFileName - Input video file name
# @input imageFileSaveArea - Folder where extracted images will be saved
# @input frameFileName - CSV file that will contain timestamp data
# @input contrastVal - contrast at which images will be changed to, determined empirically
# @input brightnessVal - increase brightness of certain videos, determined empirically
# @input cropImage - [x,y,z,w] | [x,y] is the image size, and [z,w] is the coordinate on the image where it will start creating
# the size image
###

videoFileName = ''
imageFileSaveArea = 'images' + videoFileName
frameFileName = 'frames' + videoFileName + '.csv'

In [ ]:
def determine_background(image, coord_topleft, coord_bottomright):
    total_pixels = 0
    num_white_pixels = 0

    # range through our box and check for white pixels
    for x in range(coord_topleft[0], coord_bottomright[0] + 1):
        for y in range(coord_topleft[1], coord_bottomright[1] + 1):
            curr = (x,y)
            r, g, b = image.getpixel(curr)
            if ((r,g,b) == (255,255,255)):
                num_white_pixels += 1
            total_pixels += 1
    if ((float(num_white_pixels) / total_pixels) >= 0.5):
        return False
    else:
        return True


In [ ]:
time_stamp = 0
frame = 1
indir = '../' + imageFileSaveArea
df = pd.DataFrame()

# Has the asterices appear?
firstAsterisk = False
secondAsterisk = False
thirdAsterisk = False
fourthAsterisk = False

for root, dirs, filenames in os.walk(indir):
    for f in sorted(filenames):
        
        # Read Image
        image = cv2.imread('../'+ imageFileSaveArea + '/'+f)
        im_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Preprocessing - Bilaterial Filter smoothes images to reduce noise
        bf_image = cv2.bilateralFilter(im_gray, 6, 58, 116)
        
        # Find the edges in the image and indicate if an asterisks has been found
        edges = cv2.Canny(bf_image,58,116)
        # Find contours in the image, reads shapeMask image, only looks for the external hierarchy, and uses simple approximation for contouring
        _, contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Is there an amount of astericies present?
        asterisksPresent = 0

        for cnts in contours:
            approx = cv2.approxPolyDP(cnts, 0.01*cv2.arcLength(cnts,True),True)
            area = cv2.contourArea(cnts)
            if ((len(approx) > 0) & (area > 30) or ((len(cnts) > 0) & (area > 4))):
                asterisksPresent = asterisksPresent + 1
                
        # If the first asterisk is present
        if (asterisksPresent == 1):
            firstAsterisk = True
            secondAsterisk = ''
            thirdAsterisk = ''
            fourthAsterisk = '' 
        # If the second asterisk present
        elif (asterisksPresent == 2):
            secondAsterisk = True
            firstAsterisk = ''
            thirdAsterisk = ''
            fourthAsterisk = '' 
        # If the third asterisk is present
        elif (asterisksPresent == 3):
            thirdAsterisk = True
            firstAsterisk = ''
            secondAsterisk = ''
            fourthAsterisk = ''
        # If the fourth asterisk is present
        elif (asterisksPresent == 4):
            fourthAsterisk = True
            firstAsterisk = ''
            secondAsterisk = ''
            thirdAsterisk = ''
        # If the screen is cleared
        # Don't care about 5th key (?)
        elif (asterisksPresent == 0 or asterisksPresent > 4):
            firstAsterisk = ''
            secondAsterisk = ''
            thirdAsterisk = ''
            fourthAsterisk = '' 

        print 'Asterisks present are: %d in frame: %d' % (asterisksPresent,frame)  
        
        # Checking if background is white
        log = open(os.path.join(root,f),'r')
        bg = Image.open(log)
        pix = bg.load()
        rgb_im = bg.convert('RGB')
        background_topleft = [2,100]
        background_bottomright = [5,110]
        background = determine_background(bg, background_topleft, background_bottomright)
        
        
        # First Asterisk has popped up
        if (background == False):               
            df = df.append(pd.DataFrame({'Background': background, 'First': firstAsterisk, 'Second': secondAsterisk, 'Third': thirdAsterisk, 'Fourth': fourthAsterisk, }, index=[frame]), ignore_index=False)
        else:
            df = df.append(pd.DataFrame({'Background': background, 'First': 'NaN', 'Second': 'NaN', 'Third': 'NaN', 'Fourth': 'NaN', }, index=[frame]), ignore_index=False)
        
        frame = frame + 1


In [ ]:
# Combine the timestamp csv and the data frame csv into one csv file
Time = pd.read_csv("../" + frameFileName, 
                  names = ["Time_stamp"])
result = pd.concat([df, Time], axis=1, join='inner')

outputFileForUser = 'results' + videoFileName + '.csv'


result = result.reindex(columns=['Time_stamp','Background', 'First', 'Second', 'Third', 'Fourth']).to_csv('../' + outputFileForUser, index=True)


In [ ]:
# Load new csv file and get ready to read
outputFileForUser = 'results' + videoFileName + '.csv'
nR = pd.read_csv('../' + outputFileForUser, index_col=0)


In [ ]:
# Boolean to find the first keypress and prevent from printing out every single line of the csv file to output

firstKP = False
secondKP = False
thirdKP = False
fourthKP = False
clr = False
start_time = time.time()

try:
    with open('../relevantKeyPresses' + videoFileName + '.txt', 'w') as textOutput:
        # Iterate through every row in the csv file and check specific columns for True values
        for index, row in nR.iterrows():
            x = index

            ## First Asterisk Occurance
            # Checks first column for True
            if (nR['First'].iloc[index] == True):
               # Check to see if a first keypress was already found
                if (firstKP == False):
                    # If the first keypress was not found previously, is it the same for three consecutive times?
                    isTrue = 0
                    for n in range (1,4):
                        if (nR['First'].iloc[index+n] == True):
                            isTrue = isTrue + 1
                            if (isTrue == 3):
                                # If a clear was pressed previously, modify the CLR boolean value
                                if (clr == True):
                                    clr = False
                                # Log timestamp and change the firstKP boolean to True
                                textOutput.write('%s\n' % nR.Time_stamp.iloc[[index]])
                                firstKP = True
                    

                # If the user pressed enter before the four PIN sequence is completed
                # Display time at which user presses ENTER 
                if (nR['Background'].iloc[x+1] == True and firstKP == True):
                    textOutput.write('%s\n' % nR.Time_stamp.iloc[[x+1]])
                    textOutput.write('^\n')
                    firstKP = False

            # If a first keypress was already found, and becomes missing due to a CLEAR, reset boolean
            elif (firstKP == True and pd.isnull(nR['First'].iloc[index]) == True):
                clr = True
                firstKP = False

            ## Second Asterisk Occurance
            # Check second column for True
            if (nR['Second'].iloc[index] == True):
                # Check to see if the second keypress was already found
                if (secondKP == False):
                    # If second keypress was not found previously, is it the same for three consecutive times?
                    isTrue = 0
                    for n in range(1,4):
                        if (nR['Second'].iloc[index+n] == True):
                            isTrue = isTrue + 1
                            if (isTrue == 3):
                                # If clear was pressed previously, modify the CLR boolean value
                                if (clr == True):
                                    clr = False
                                textOutput.write('%s\n' % nR.Time_stamp.iloc[[index]])
                                secondKP = True

                # If the user pressed enter before the four PIN sequence is completed
                # Display time at which the user has press ENTER
                if (nR['Background'].iloc[x+1] == True):
                    textOutput.write('%s\n' % nR.Time_stamp.iloc[[x+1]])
                    textOutput.write('^\n')
                    firstKP = False
                    secondKP = False

            # If a second keypress was found, and becomes missing due to a CLEAR, reset boolean
            elif (secondKP == True and pd.isnull(nR['Second'].iloc[index]) == True):
                secondKP = False
                clr = True

            ## Third Asterisk Occurance
            # Check third column for True
            if (nR['Third'].iloc[index] == True):
                # Check to see if the third keypress was already found
                if (thirdKP == False and fourthKP == False):
                    # If the third keypress was not found and the fourth keypress is also not found because the 
                    # asterisks are put in chronological order, check the next three to see if it is also true
                    isTrue = 0
                    for n in range(1,4):
                        if (nR['Third'].iloc[index+n] == True):
                            isTrue = isTrue + 1
                            if (isTrue == 3):
                                if (clr == True):    
                                    clr = False
                                textOutput.write('%s\n' % nR.Time_stamp.iloc[[index]])
                                thirdKP = True

                # If the user presses enter before the four PIN sequence is completed.
                # Log the timestamp at which they pressed enter
                if (nR['Background'].iloc[x+1] == True):
                    nR.Time_stamp.iloc[[x+1]]
                    textOutput.write('%s\n' % nR.Time_stamp.iloc[[x+1]])
                    textOutput.write('^\n')
                    firstKP = False
                    secondKP = False
                    thirdKP = False

            # If a third keypress was found, and becomes missing due to a CLEAR, reset boolean
            elif (thirdKP == True and pd.isnull(nR['Fourth'].iloc[index]) == True):
                thirdKP = False
                clr = True

            ## Fourth Asterisk Occurance
            # Check fourth column for True
            # Check to see if fourth keypress was found
            if (nR['Fourth'].iloc[index] == True):

                # If a clear was not taken place, display time at which the fourth asterisk had appeared
                if (fourthKP == False and thirdKP == True):
                    # Again... Is it this fourth keypress consistent? Check for the next three ahead
                    isTrue = 0
                    for n in range (1,4):
                        if (nR['Fourth'].iloc[index+n] == True):
                            isTrue = isTrue + 1
                            if (isTrue == 3):
                                if (clr == True):
                                    clr = False
                                textOutput.write('%s\n' % nR.Time_stamp.iloc[[index]])
                                fourthKP = True

            # Check to see if a key was removed
            # Due to high frame rates, sometimes the objects do not dissapear simulataneously.
            elif (pd.isnull(nR['Fourth'].iloc[index]) == True and fourthKP == True):
                # If the third column is considered true, is it true for a certain duration?
                if (nR['Third'].iloc[index] == True):
                    isTrue = 0
                    for n in range(1,4):
                        if (nR['Third'].iloc[index+n] == True):
                            isTrue = isTrue + 1
                            # If it is true, make the fourth keypress boolean false and find a new fourth keypress
                            if (isTrue == 3):
                                fourthKP = False
                                clr = True
                            # If it fails to be true for three consecutive times, this indicates that the screen is actually clearing
                            else:
                                firstKP = False
                                secondKP = False
                                thirdKP = False
                                fourthKP = False
                                clr = False

                                textOutput.write('%s\n' % nR.Time_stamp.iloc[[index]])
                                textOutput.write('---------------------------------------\n')
                
                # If all the columns of the first/second/third/fourth keypress are null, the screen has cleared.
                # Log this information.
                else:
                    # Once all keys were found and the screen clears, reset all booleans
                    firstKP = False
                    secondKP = False
                    thirdKP = False
                    fourthKP = False
                    clr = False
                    
                    textOutput.write('%s\n' % nR.Time_stamp.iloc[[index]])
                    textOutput.write('---------------------------------------\n')
            
except:
    print 'End of File'
    
    
with open('../relevantKeyPresses' + videoFileName + '.txt', 'r') as input:
    with open('../cleanedRelevantKeyPresses' + videoFileName + '.txt','wb') as output: 
        for line in input:
            if (line!="Name: Time_stamp, dtype: float64"+"\n"):
                output.write(line)
print 'Complete'
print "My program took", time.time() - start_time, "to run"

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
####
# This is for debugging purposes and checking the edges of a specific image if there are problems with reading
# the csv file.

## Problems:
# Don't know how to calculate/account for the bilateral filter parameters
####

#Bilateral filtering
image = cv2.imread('../images106a/')
im_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

bf_image = cv2.bilateralFilter(im_gray, 6, 175, 175)
ed_image = cv2.Canny(im_gray, 58, 116)

_, contours, _ = cv2.findContours(ed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

asterisksPresent = 0
for cnts in contours:
    approx = cv2.approxPolyDP(cnts, 0.01*cv2.arcLength(cnts,True),True)
    area = cv2.contourArea(cnts)
    if ((len(approx) > 0) & (area > 30) or ((len(cnts) > 0) & (area > 4))):
        asterisksPresent = asterisksPresent + 1
        
print 'Asterisks present are: %d' % asterisksPresent        
cv2.drawContours(ed_image, contour_list,  -1, (255,255,0), 2)
plt.imshow(ed_image)
plt.show()

In [ ]:
###
# Calculate the threshold needed for the canny edge filter
###
high_thresh, thresh_im = cv2.threshold(bf_image, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
lowThresh = 0.5*high_thresh

print high_thresh
print lowThresh